In [77]:
import pandas as pd
import altair as alt
import re

In [78]:
matches =  '/Users/anton/sandbox/vgp/viper/lrtomt.tsv'
orfs =     '/Users/anton/sandbox/vgp/viper/orfs.faa'
gene_faa = '/Users/anton/sandbox/vgp/lrtomt.faa'
genome =   '/Users/anton/sandbox/vgp/viper/rVipLat1.pri.cur.20211208.fasta'

In [79]:
df = pd.read_csv(
    matches,
    sep='\t',names='Q,T,id,L,mis,gap,qStart,qEnd,tStart,tEnd,E,B'.split(',')
)

In [80]:
df.head()

,Q,T,id,L,mis,gap,qStart,qEnd,tStart,tEnd,E,B
0,ENST00000307198.11,SUPER_4_ORF.652106_1,62.5,104,39,0,186,289,46,149,4.920000e-37,134.0
1,ENST00000307198.11,SUPER_4_ORF.3808_1,60.5,114,45,0,7,120,7,120,1.110000e-35,130.0
2,ENST00000307198.11,SUPER_4_ORF.652104_1,60.6,66,26,0,121,186,88,153,5.460000e-17,82.0
3,ENST00000307198.11,SUPER_14_ORF.182197_1,52.7,55,26,0,120,174,18,72,2.890000e-10,61.2
4,ENST00000643715.1,SUPER_4_ORF.327979_1,59.2,71,29,0,28,98,31,101,2.210000e-21,89.4


In [81]:
df['chr'] = df['T'].str.extract('([\w\d]+)_')

In [82]:
df['orf'] = df['T'].str.extract('[\w\d]+_(ORF.+)')

In [83]:
orf_data = {
    'chr':[],
    'orf':[],
    'start':[],
    'end':[],
    'strand':[],
    'type':[],
    'orf_len':[],
    'frame':[],
    'sCod':[],
    'eCod':[]
}

mapping = dict(zip(range(1,11),orf_data.keys()))
ids = df['T'].to_list()

info = re.compile('^>([\w\d]+)_(ORF\.\d+_\d+)\s\[(\d+)-(\d+)\]\(([\+-])\)\stype:(\w+)\slength:(\d+)\sframe:(-{0,1}\d+)\sstart:(\w+)\sstop:(\w+)')
with open(orfs,'r') as f:
    for line in f:
        if line.startswith('>') and ( line.split(' ')[0][1:] in ids ):
            try:
                result = info.match(line)
                for i in range(1,11):
                    orf_data[mapping[i]].append(result.group(i))
            except:
                print(line)

In [84]:
gene_data = {
    'Q':[],
    'gene_len':[]
}

id_len = re.compile('>([\w\_\.\(\)\s]+)\slength=(\d+)')
with open(gene_faa,'r') as f:
    for line in f:
        if line.startswith('>'):
            try:
                result = id_len.match(line)
                gene_data['Q'].append(result.group(1))
                gene_data['gene_len'].append(result.group(2))
            except:
                print(line)

In [85]:
seq_id = ""

genome_data = { 
    'chr':[],
    'chr_len':[]
}
               

with open(genome,'r') as f:
    for line in f:
        line = line.rstrip()
        if line.startswith('>'):
            if seq_id != "":
                genome_data['chr'].append(seq_id)
                genome_data['chr_len'].append(count)
            seq_id = line[1:]
            count = 0
        else:
            count += len(line)-1
    genome_data['chr'].append(seq_id)
    genome_data['chr_len'].append(count)

In [86]:
genome_data = pd.DataFrame.from_dict(genome_data)

In [87]:
gene_data = pd.DataFrame.from_dict(gene_data)

In [88]:
gene_data

,Q,gene_len
0,ENST00000307198.11,291
1,ENST00000643715.1,156


In [89]:
orf_data = pd.DataFrame.from_dict(orf_data)

In [90]:
hmap = df['Q,id,L,mis,gap,qStart,qEnd,chr,orf'.split(',')]

In [91]:
hmap.head()

,Q,id,L,mis,gap,qStart,qEnd,chr,orf
0,ENST00000307198.11,62.5,104,39,0,186,289,SUPER_4,ORF.652106_1
1,ENST00000307198.11,60.5,114,45,0,7,120,SUPER_4,ORF.3808_1
2,ENST00000307198.11,60.6,66,26,0,121,186,SUPER_4,ORF.652104_1
3,ENST00000307198.11,52.7,55,26,0,120,174,SUPER_14,ORF.182197_1
4,ENST00000643715.1,59.2,71,29,0,28,98,SUPER_4,ORF.327979_1


In [92]:
hmap = hmap.merge(orf_data[['orf','orf_len','strand','start','end']],how='left')

In [93]:
hmap = hmap.merge(gene_data,how='left')

In [94]:
hmap = hmap.merge(genome_data,how='left')

In [95]:
hmap

,Q,id,L,mis,gap,qStart,qEnd,chr,orf,orf_len,strand,start,end,gene_len,chr_len
0,ENST00000307198.11,62.5,104,39,0,186,289,SUPER_4,ORF.652106_1,459,+,1301957,1302416,291,123576131
1,ENST00000307198.11,60.5,114,45,0,7,120,SUPER_4,ORF.3808_1,462,+,1299624,1300086,291,123576131
2,ENST00000307198.11,60.6,66,26,0,121,186,SUPER_4,ORF.652104_1,480,+,1301147,1301627,291,123576131
3,ENST00000307198.11,52.7,55,26,0,120,174,SUPER_14,ORF.182197_1,222,-,7588892,7589114,291,13689649
4,ENST00000643715.1,59.2,71,29,0,28,98,SUPER_4,ORF.327979_1,420,+,1260955,1261375,156,123576131
5,ENST00000643715.1,58.0,50,21,0,97,146,SUPER_4,ORF.3698_1,330,+,1265718,1266048,156,123576131


In [96]:
hmap['gene_len']=hmap['gene_len'].astype('int')
hmap['chr_len']=hmap['chr_len'].astype('int')
hmap['start']=hmap['start'].astype('int')
hmap['end']=hmap['end'].astype('int')

In [97]:
hmap

,Q,id,L,mis,gap,qStart,qEnd,chr,orf,orf_len,strand,start,end,gene_len,chr_len
0,ENST00000307198.11,62.5,104,39,0,186,289,SUPER_4,ORF.652106_1,459,+,1301957,1302416,291,123576131
1,ENST00000307198.11,60.5,114,45,0,7,120,SUPER_4,ORF.3808_1,462,+,1299624,1300086,291,123576131
2,ENST00000307198.11,60.6,66,26,0,121,186,SUPER_4,ORF.652104_1,480,+,1301147,1301627,291,123576131
3,ENST00000307198.11,52.7,55,26,0,120,174,SUPER_14,ORF.182197_1,222,-,7588892,7589114,291,13689649
4,ENST00000643715.1,59.2,71,29,0,28,98,SUPER_4,ORF.327979_1,420,+,1260955,1261375,156,123576131
5,ENST00000643715.1,58.0,50,21,0,97,146,SUPER_4,ORF.3698_1,330,+,1265718,1266048,156,123576131


In [98]:
hmap['qS_norm']=hmap['qStart']/hmap['gene_len']
hmap['qE_norm']=hmap['qEnd']/hmap['gene_len']
hmap['gS_norm']=hmap['start']/hmap['chr_len']
hmap['gE_norm']=hmap['end']/hmap['chr_len']

In [99]:
charts = []
for name, group in hmap.groupby(['Q','chr']):
    facet = {
        'view':[],
        'start':[],
        'end':[],
        'block':[],
        'id':[]
    }
    for i,row in enumerate(group.iterrows()):
        for view in ['tr','orf','chr']:
            facet['view'].append(view)
            facet['block'] = i
            facet['id'].append(row[1]['id'])
            if view == 'tr':
                facet['start'].append(row[1]['qS_norm'])
                facet['end'].append(row[1]['qE_norm'])
            elif view == 'orf':
                l = group['end'].max()-group['start'].min()
                facet['start'].append((row[1]['start']-group['start'].min())/l)
                facet['end'].append((row[1]['end']-group['start'].min())/l)
            elif view == 'chr':
                facet['start'].append(row[1]['gS_norm'])
                facet['end'].append(row[1]['gE_norm'])
    t = pd.DataFrame.from_dict(facet) 
    t['mid']=t['start']+(t['end']-t['start'])/2
    
    base = alt.Chart(t).encode(
        x = alt.X('start',title=None),
        y = alt.Y('view',title=None),
    ).properties(
        title='/'.join(name)
    )
    bars = base.mark_bar(opacity=.75,stroke='black').encode(
        x2 = alt.X2('end:Q',title=None),
        color=alt.Color(
            'id',
            scale=alt.Scale(
                scheme="goldred"
            )
        )
    )
    line1 = alt.Chart(t[(t['view']=='orf') | (t['view']=='tr')]).mark_line(size=.5,color='black').encode(
        x='mid',
        y = 'view',
        order = 'block'
    )
    
    line2 = alt.Chart(t[(t['view']=='orf') | (t['view']=='chr')]).mark_line(size=.5,color='black').encode(
        x='mid',
        y = 'view',
        order = 'block'
    )
    
    point = alt.Chart(t[(t['view']=='chr')]).mark_point(color='red').encode(
        x='mid',
        y = 'view',
        order = 'block'
    )
    
    charts.append((bars + line1 + line2 + point).properties(
        width=1000,
        height=75
    ).interactive())
    

In [100]:
alt.vconcat(*charts)

alt.VConcatChart(...)